In [20]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import json
import requests

QUOTES_URL = 'https://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities/{security}.json' \
                  '?from={start}&till={end}&iss.meta=off'

DIVIDENDS_URL = 'https://iss.moex.com/iss/securities/{security}/dividends.json?iss.meta=off'

def retrieve_quotes(security, start, end):
    url = QUOTES_URL.format(
    security = security,
    start = start,
    end = end)
    r = requests.get(url)
    data = pd.read_json(json.dumps(r.json()['history']), orient = 'split', typ = 'frame')
    data['TRADEDATE'] = data['TRADEDATE'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    return data

def retrieve_dividends(security):
    url = DIVIDENDS_URL.format(security = security)
    r = requests.get(url)
    data = pd.read_json(json.dumps(r.json()['dividends']), orient = 'split', typ = 'frame')
    data['registryclosedate'] = data['registryclosedate'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    return data

def merge_quotes(security, start, end):
    dt_start = datetime.strptime(start, '%Y-%m-%d')
    dt_end = datetime.strptime(end, '%Y-%m-%d')
    quotes = retrieve_quotes(security, start, end)
    last = quotes['TRADEDATE'].to_list()[-1]
    while (dt_end - last).days > 5:
        dt_start = last + timedelta(days = 1)
        start = datetime.strftime(dt_start, '%Y-%m-%d')
        tmp = retrieve_quotes(security, start, end)
        quotes = pd.concat([quotes, tmp])
        last = quotes['TRADEDATE'].to_list()[-1]
    quotes.reset_index(inplace = True)
    quotes.drop(columns = ['index'], inplace = True)
    quotes.set_index('TRADEDATE', inplace = True)
    return quotes

def validate_dividends(divs, quotes):
    new_vals = []
    for i in range(len(divs)):
        date = divs.iloc[i]['registryclosedate']
        pre_div = quotes.loc[:date - timedelta(days = 2)].iloc[-1]['HIGH']
        post_div = quotes.loc[date - timedelta(days = 1):].iloc[0]['LOW']
        if (pre_div - post_div) < (divs.iloc[i]['value'] / 2):
            new_vals.append(0)
        else:
            new_vals.append(divs.iloc[i]['value'])
    divs['value'] = new_vals
    
def calculate_adjusted_prices(divs, quotes):
    price_mults = np.ones(len(quotes))
    for i in range(len(divs)):
        date = divs.iloc[i]['registryclosedate']
        close = quotes.loc[:date - timedelta(days = 2)].iloc[-1]['CLOSE']
        new_mult = (close - divs.iloc[i]['value']) / close
        price_mults *= np.where(quotes.index >= 
                           (divs.iloc[i]['registryclosedate'] - timedelta(days = 1)), 1, new_mult)
    quotes['ADJ_CLOSE'] = quotes['CLOSE'] * price_mults
    
def get_data(security, start, end):
    divs = retrieve_dividends(security)
    data = merge_quotes(security, start, end)
    validate_dividends(divs, data)
    calculate_adjusted_prices(divs, data)
    return data, divs

In [21]:
data, divs = get_data('GAZP', '2014-01-01', '2023-08-10')

In [22]:
import plotly.graph_objects as go

fig = go.Figure()

fig.update_layout(title = 'GAZP')
fig.add_trace(go.Scatter(x = data.index, y = data['CLOSE'],
                    mode='lines',
                    name='Price'))
fig.add_trace(go.Scatter(x = data.index, y = data['ADJ_CLOSE'],
                    mode='lines',
                    name='Adj Price'))
fig.add_trace(go.Bar(x = divs['registryclosedate'], y = divs['value'],
                    name = 'Dividends',
                    text = divs['value'], 
                    textposition = 'outside'))

fig.show()